In [34]:
# %pip install xlrd
import pandas as pd
import numpy as np

df = pd.read_excel("/Users/snow/Documents/IRON HACK COURSE/Week 2/Mini Project/GSAF5.xls")
print(df.shape)
print(df.nunique())
print(df['Type'].unique(), '\n')
# print(df['Sex'].unique(), '\n')
# for element in df['Activity'].unique():
#     print(element)

df.head()

# print("The categorical columns are the following:'Type, 'Country', 'State', 'Activity', 'Sex', 'Age', 'Species', 'Time'")
categorical_cols = ['Type', 'Country', 'State', 'Activity', 'Sex', 'Age', 'Fatal Y/N', 'Species ', 'Time']
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")
print("-----------------------------------------------------------------------------------")
df.replace({'Sex': {'F ': 'F', 'M ': 'M', ' M': 'M', 'm': 'M', 'lli': 'NaN', 'M x 2': 'M', 'N': 'NaN', '': 'NaN', '.': 'NaN'}}, inplace=True)
print(df['Sex'].unique(), '\n')
print(df['Sex'].value_counts())






(7045, 23)
Date              6089
Year               261
Type                13
Country            247
State              937
Location          4595
Activity          1606
Name              5770
Sex                 10
Age                250
Injury            4162
Fatal Y/N           12
Time               460
Species           1724
Source            5384
pdf               6789
href formula      6784
href              6776
Case Number       6777
Case Number.1     6775
original order    6797
Unnamed: 21          1
Unnamed: 22          2
dtype: int64
['Unprovoked' 'Provoked' 'Questionable' 'unprovoked' ' Provoked'
 'Watercraft' 'Sea Disaster' nan '?' 'Unconfirmed' 'Unverified' 'Invalid'
 'Under investigation' 'Boat'] 

Type: 13 unique values
Country: 247 unique values
State: 937 unique values
Activity: 1606 unique values
Sex: 10 unique values
Age: 250 unique values
Fatal Y/N: 12 unique values
Species : 1724 unique values
Time: 460 unique values
---------------------------------------------